In [3]:
import pandas as pd
import numpy as np
import os, sys

In [4]:
d = '../../../code/cedar_icerfire/data/'
list(filter(lambda x: 'pepx' in x.lower() or 'aligned' in x.lower(), os.listdir(d)))

['pepx', '230829_sine_processed_pepx_mutations.csv', 'aligned_icore']

In [10]:
%ls {d}aligned_icore/

230125_cedar_aligned_icore.csv   230308_prime_aligned_icore.csv
230125_ibel_aligned_icore.csv    230418_cedar_aligned.csv
230125_nepdb_aligned_icore.csv   230418_nepdb_aligned.csv
230125_prime_aligned_icore.csv   230418_prime_aligned.csv
230308_cedar_aligned_icore.csv   230717_cedar_aligned_icore.csv
230308_ibel_aligned_icore.csv    backup/
230308_nepdb_aligned_icore.csv   backup_230308_beforeForeignness/


In [9]:
%ls {d}pepx/

230125_cedar_aligned_pepx_fold.csv      230308_prime_aligned_pepx.csv
230125_cedar_related_pepx_fold.csv      230418_cedar_aligned_pepx.csv
230125_ibel_aligned_pepx.csv            230418_nepdb_aligned_pepx.csv
230125_ibel_normal_pepx.csv             230418_prime_aligned_pepx.csv
230125_nepdb_aligned_pepx.csv           2308XX_cedar_wildtype.csv
230125_nepdb_normal_pepx.csv            2308XX_cedar_wildtype_pepx_output.csv
230125_prime_aligned_pepx.csv           2308XX_cedar_wt_icores.csv
230125_prime_related_pepx.csv           2308XX_cedar_wt_icores_pepx_output.csv
230308_cedar_aligned_pepx_fold.csv      TCGA_PANCAN/
230308_ibel_aligned_pepx.csv            backup/
230308_nepdb_aligned_pepx.csv


In [11]:
# aligned icores first

cedar = pd.read_csv(f'{d}aligned_icore/230418_cedar_aligned.csv')
prime = pd.read_csv(f'{d}aligned_icore/230418_prime_aligned.csv')
nepdb = pd.read_csv(f'{d}aligned_icore/230418_nepdb_aligned.csv')

In [17]:
cedar.drop(columns=[x for x in cedar.columns if 'unnamed' in x.lower() or 'vs' in x.lower() or 'improve' in x.lower() or 'mixed_' in x.lower() or '_input' in x.lower()]).columns

Index(['HLA', 'agg_label', 'fold', 'Peptide', 'wild_type', 'icore_mut',
       'core_mut', 'len', 'EL_rank_mut', 'EL_rank_wt', 'trueHLA_EL_rank',
       'icore_mut_start', 'len_icore_mut', 'icore_wt_aligned',
       'icore_dissimilarity_score', 'icore_len_mut', 'icore_len_wt',
       'icore_anchor', 'icore_mutation_position', 'icore_anchor_mutation',
       'icore_mut_score', 'icore_blsm_mut_score', 'aliphatic_index', 'boman',
       'hydrophobicity', 'isoelectric_point', 'VHSE1', 'VHSE3', 'VHSE7',
       'VHSE8', 'seq_id', 'core_wt_aligned', 'EL_rank_wt_aligned',
       'ratio_rank', 'foreignness_score'],
      dtype='object')